# Artificial Intelligence 
## Machine Translation Project Dutch to Italian
In this notebook, I tried out my own machine translation project using memories I have worked on in the past.
I used a previously coded project to write down the code for preparation, models and prediction.


In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.models import Sequential
import re

C:\Users\France\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17080963416053755903
]


## Dataset
The dataset is created from a translation memory in Trados format. The memory files has been made anonimous and all private data have been deleted. The format of the memory files contains all data and tags which have been elaborated into a common, simple dataset for the two languages.
### Load Data
The data is located in `data/testfileIT` and `data/testfileNL`. The `testfileIT` file contains Italian translation of the  source sentences in the `testfileNL` file. The code below has been run on the original bilingual file to create the two sets.

In [30]:


memoryfile=[]
ita_file=[]
nl_file=[]
 # Remove HTML tags using RegEx
pattern = re.compile(r'<ut>.*?</ut>')  # tags between <ut></ut>

with open('data/NL-IT.txt', encoding="utf8") as f:
    memoryfile = [line.rstrip('\n') for line in f]
    
              
ita_final = open('data/testfileIT', 'w+', encoding="utf8")
nl_final = open('data/testfileNL', 'w+', encoding="utf8")
for t in range(len(memoryfile)):
    if memoryfile[t].startswith('<tuv lang="IT-IT">'):
        t=t+1
        unit=memoryfile[t]
        #print(unit)
        unit=pattern.sub('', unit) # delete internal tags
        unit=unit.strip('</seg>') #delete starting ending seg tag
       # print(unit)
        unit.lower()
        ita_final.writelines(unit + '\n')
        #print(unit.encode('utf8'))
        ita_file.append(unit)
    if memoryfile[t].startswith('<tuv lang="NL-NL">'):
        t=t+1
        unit=memoryfile[t]     
        unit=pattern.sub('', unit) # delete internal tags
        unit=unit.strip('</seg>') #delete starting ending seg tag
        unit.lower()
        nl_final.writelines(unit + '\n')
        nl_file.append(unit)
           

ita_final.close()
nl_final.close()


Load the Dutch and Italian data from these files from running the cell below.

In [32]:
# Load dutch data
#dutch_sentences = helper.load_data('data/testfileNL', encoding="utf8")
# Load italian data
#italian_sentences = helper.load_data('data/testfileIT', encoding="utf8")
import os
bron= []
doel = []
with open("data/testfileNL", encoding="utf8") as f:
    text=f.read()
    print(len(text))
    bron=text.split('\n')
    
with open("data/testfileIT", encoding="utf8") as f:
    text=f.read()
    print(len(text))
    doel=text.split('\n')
    



print(len(bron))
print(len(doel))
print(bron[27618])
print(doel[27618])
print(type(doel))
print('Dataset Loaded')

2650018
2821810
27619
27619


<class 'list'>
Dataset Loaded


### Files
Each line in `small_vocab_en` contains an Dutch sentence with the respective translation in each line.  View the first two lines from each file.

In [33]:
for sample_i in range(2):
    print('small_vocab_nl Line {}:  {}'.format(sample_i + 1, bron[sample_i]))
    print('small_vocab_it Line {}:  {}'.format(sample_i + 1, doel[sample_i]))

small_vocab_nl Line 1:  Overwegende dat bij het lossen van de grondstoffen per schip gebruik wordt gemaakt van filtertrechters en overdekte transportbanden;
small_vocab_it Line 1:  In considerazione del fatto che per lo scarico delle materie prime giunte per nave viene fatto uso di un imbuto con filtro e nastri di trasporto coperti;
small_vocab_nl Line 2:  Ruim en comfortabel, pal aan een breed zandstrand.
small_vocab_it Line 2:  Spazioso e confortevole, prospiciente un’ampia spiaggia.


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [46]:


dutch_words_counter = collections.Counter([word for sentence in bron for word in sentence.split()])
italian_words_counter = collections.Counter([word for sentence in doel for word in sentence.split()])

print('{} Dutch words.'.format(len([word for sentence in bron for word in sentence.split()])))
print('{} unique Dutch words.'.format(len(dutch_words_counter)))
print('10 Most common words in the Dutch dataset:')
print('"' + '" "'.join(list(zip(*dutch_words_counter.most_common(100)))[0]) + '"')
print()
print('{} Italian words.'.format(len([word for sentence in doel for word in sentence.split()])))
print('{} unique Italian words.'.format(len(italian_words_counter)))
print('10 Most common words in the Italian dataset:')
print('"' + '" "'.join(list(zip(*italian_words_counter.most_common(100)))[0]) + '"')

392958 Dutch words.
48852 unique Dutch words.
10 Most common words in the Dutch dataset:
"de" "van" "en" "het" "een" "in" "te" "is" "op" "met" "dat" "xx" "voor" "of" "aan" "De" "die" "niet" "door" "zijn" "deze" "worden" "bij" "om" "je" "als" "tot" "heeft" "wordt" "ook" "dan" "kan" "Het" "yy" "u" "dit" "uit" "hebben" "er" "geen" "-" "naar" "/xx" "zij" "over" "meer" "kunnen" "zich" "partijen" "c.s." "onder" "uw" "zal" "alle" "•" "haar" "the" "In" "nog" "hun" "na" "schade" "andere" "1" "€" "maar" "per" "font=&quot;Tahoma&quot;" "and" "en/of" "ten" "complexscriptsfont=&quot;Tahoma&quot;" "Italiaanse" "al" "size=&quot;10&quot;" "Dit" "moet" "zou" "was" "&lt;cf" "dient" "Een" "tussen" "2" "Deze" "to" "zoals" "–" "wel" "ze" "nieuwe" "eigen" "eerste" "binnen" "goed" "Voor" "indien" "koe" "klanten" "recht"

415103 Italian words.
47288 unique Italian words.
10 Most common words in the Italian dataset:
"di" "e" "il" "in" "la" "per" "che" "a" "del" "un" "è" "non" "le" "xx" "della" "con" "i" "una" 

For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `dutch_sentences` and `italian_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [47]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)

    return x_tk.texts_to_sequences(x), x_tk
#run tests
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'brown': 3, 'fox': 4, 'jumps': 5, 'over': 6, 'lazy': 7, 'dog': 8, 'a': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 3, 4, 5, 6, 1, 7, 8]
Sequence 2 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 3, 4, 5, 6, 1, 7, 8]
Sequence 3 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 9, 17]
Sequence 4 in x
  Input:  This is a short sentence .
  Output: [18, 19, 9, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the dutch sequences have the same length and all the italian sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [48]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length=max([len(sentence) for sentence in x])
        
        print(length)
    return pad_sequences(x, maxlen=length, padding='post')
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

6
10
Sequence 1 in x
  Input:  [1 2 3 4 5 6 1 7 8]
  Output: [1 2 3 4 5 6 1 7 8 0]
Sequence 2 in x
  Input:  [1 2 3 4 5 6 1 7 8]
  Output: [1 2 3 4 5 6 1 7 8 0]
Sequence 3 in x
  Input:  [10 11 12  2 13 14 15 16  9 17]
  Output: [10 11 12  2 13 14 15 16  9 17]
Sequence 4 in x
  Input:  [18 19  9 20 21]
  Output: [18 19  9 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [53]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_dutch_sentences, preproc_italian_sentences, dutch_tokenizer, italian_tokenizer =\
    preprocess(bron, doel)
    
max_dutch_sequence_length = preproc_dutch_sentences.shape[1]
max_italian_sequence_length = preproc_italian_sentences.shape[1]
dutch_vocab_size = len(dutch_tokenizer.word_index)+1
italian_vocab_size = len(italian_tokenizer.word_index)+1

print('Data Preprocessed')
print("Max Dutch sentence length:", max_dutch_sequence_length)
print("Max Italian sentence length:", max_italian_sequence_length)
print("Dutch vocabulary size:", dutch_vocab_size)
print("Italian vocabulary size:", italian_vocab_size)

410
414
Data Preprocessed
Max Dutch sentence length: 410
Max Italian sentence length: 414
Dutch vocabulary size: 27887
Italian vocabulary size: 25658


In [ ]:
len(preproc_italian_sentences[3])

In [39]:
preproc_dutch_sentences

array([[1104,   14,   25, ...,    0,    0,    0],
       [1452,    3, 4022, ...,    0,    0,    0],
       [  12,  139, 2180, ...,    0,    0,    0],
       ...,
       [  21,  687,   17, ...,    0,    0,    0],
       [ 121,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0]])

In [40]:
max_italian_sequence_length

414

In [41]:
max_dutch_sequence_length

410

## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the italian translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the italian translation.  You'll be using this function to better understand the output of the neural network.

In [54]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates dutch to italian.

In [52]:
def simple_model(input_shape, output_sequence_length, dutch_vocab_size, italian_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param dutch_vocab_size: Number of unique dutch words in the dataset
    :param italian_vocab_size: Number of unique italian words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
   
    rnn = GRU(64, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(italian_vocab_size))(rnn)
   
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    model.get_config()
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_dutch_sentences, max_italian_sequence_length)
print(tmp_x.shape)
print(preproc_italian_sentences.shape)
print(preproc_italian_sentences.shape[-2])
tmp_x = tmp_x.reshape((-1, preproc_italian_sentences.shape[-2], 1))
#print(len(tmp_x))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_italian_sequence_length,
    dutch_vocab_size,
    italian_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_italian_sentences, batch_size=56, epochs=10, validation_split=0.2)

# Print prediction(s)


(27619, 414)
(27619, 414, 1)
414
Train on 22095 samples, validate on 5524 samples
Epoch 1/10


KeyboardInterrupt: 

In [381]:
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], italian_tokenizer))

e e e e e e e <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [382]:
def embed_model(input_shape, output_sequence_length, dutch_vocab_size, italian_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param dutch_vocab_size: Number of unique dutch words in the dataset
    :param italian_vocab_size: Number of unique italian words in the dataset
    :return: Keras model built, but not trained
    """
    batch = 32
    learning_rate = 1e-3
    

    rnn = GRU(batch, return_sequences=True)
    embedding=Embedding(italian_vocab_size, batch, input_length=input_shape[1])
    logits = TimeDistributed(Dense(italian_vocab_size))
    
    model = Sequential()
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    
    # TODO: Implement
    return model
tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(preproc_dutch_sentences, max_italian_sequence_length)
print(tmp_x.shape)
tmp_x = tmp_x.reshape((-1, preproc_italian_sentences.shape[-2]))


# TODO: Train the neural network
model_with_embdding= embed_model(tmp_x.shape, max_italian_sequence_length, dutch_vocab_size, italian_vocab_size)
model_with_embdding.fit(tmp_x, preproc_italian_sentences, batch_size=128, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(model_with_embdding.predict(tmp_x[:1])[0], italian_tokenizer))


C:\Users\France\Anaconda3\lib\site-packages\keras\engine\sequential.py:109: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


(255, 122)
Train on 204 samples, validate on 51 samples
Epoch 1/10
204/204 [==============================] - 16s 79ms/step - loss: 7.3804 - acc: 4.8216e-04 - val_loss: 5.7528 - val_acc: 0.0235
Epoch 2/10
204/204 [==============================] - 1s 5ms/step - loss: 5.7107 - acc: 0.3056 - val_loss: 4.4914 - val_acc: 0.8356
Epoch 3/10
204/204 [==============================] - 1s 5ms/step - loss: 4.5244 - acc: 0.7796 - val_loss: 3.6779 - val_acc: 0.8428
Epoch 4/10
204/204 [==============================] - 1s 5ms/step - loss: 3.7918 - acc: 0.7866 - val_loss: 3.1258 - val_acc: 0.8488
Epoch 5/10
204/204 [==============================] - 1s 5ms/step - loss: 3.3198 - acc: 0.7905 - val_loss: 2.7972 - val_acc: 0.8520
Epoch 6/10
204/204 [==============================] - 1s 5ms/step - loss: 3.0579 - acc: 0.7921 - val_loss: 2.5237 - val_acc: 0.8531
Epoch 7/10
204/204 [==============================] - 1s 5ms/step - loss: 2.8151 - acc: 0.7929 - val_loss: 2.2101 - val_acc: 0.8541
Epoch 8/10
204

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [383]:
# TODO: Print prediction(s)
print(logits_to_text(model_with_embdding.predict(tmp_x[:1])[0], italian_tokenizer))

<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [384]:
def bd_model(input_shape, output_sequence_length, dutch_vocab_size, italian_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param dutch_vocab_size: Number of unique dutch words in the dataset
    :param italian_vocab_size: Number of unique italian words in the dataset
    :return: Keras model built, but not trained
    """
    batch = 256
    learning_rate = 1e-3

    rnn = GRU(batch, return_sequences=True, dropout=0.1)
    bidirectional=Bidirectional(rnn, input_shape = input_shape[1:])
    logits = TimeDistributed(Dense(italian_vocab_size, activation='softmax'))
    
    model = Sequential()
    model.add(bidirectional)
    model.add(logits)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_bd_model(bd_model)
print('tested')


# TODO: Reshape the input
print(preproc_italian_sentences.shape[1])
tmp_x = pad(preproc_dutch_sentences, preproc_italian_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_italian_sentences.shape[-2], 1))


# TODO: Train the neural network
bidirectional_model= bd_model(tmp_x.shape, preproc_italian_sentences.shape[1], dutch_vocab_size, italian_vocab_size)
bidirectional_model.fit(tmp_x, preproc_italian_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(bidirectional_model.predict(tmp_x[:1])[0], italian_tokenizer))

C:\Users\France\Anaconda3\lib\site-packages\keras\engine\sequential.py:109: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


tested
122
Train on 204 samples, validate on 51 samples
Epoch 1/10
204/204 [==============================] - 27s 135ms/step - loss: 7.3491 - acc: 0.5284 - val_loss: 7.2835 - val_acc: 0.8346
Epoch 2/10
204/204 [==============================] - 11s 52ms/step - loss: 7.2839 - acc: 0.7771 - val_loss: 7.2157 - val_acc: 0.8545
Epoch 3/10
204/204 [==============================] - 11s 53ms/step - loss: 7.2067 - acc: 0.7913 - val_loss: 7.1387 - val_acc: 0.8550
Epoch 4/10
204/204 [==============================] - 11s 53ms/step - loss: 7.1224 - acc: 0.7926 - val_loss: 7.0644 - val_acc: 0.8550
Epoch 5/10
204/204 [==============================] - 11s 53ms/step - loss: 7.0556 - acc: 0.7930 - val_loss: 6.9973 - val_acc: 0.8550
Epoch 6/10
204/204 [==============================] - 11s 53ms/step - loss: 6.9824 - acc: 0.7930 - val_loss: 6.9320 - val_acc: 0.8550
Epoch 7/10
204/204 [==============================] - 11s 53ms/step - loss: 6.9246 - acc: 0.7930 - val_loss: 6.8524 - val_acc: 0.8550
Epoch

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [285]:
def encdec_model(input_shape, output_sequence_length, dutch_vocab_size, italian_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param dutch_vocab_size: Number of unique dutch words in the dataset
    :param italian_vocab_size: Number of unique italian words in the dataset
    :return: Keras model built, but not trained
    """
    

    batch = 128
    learning_rate = 1e-2

    rnn = GRU(batch, return_sequences=True)
    repeatVector=RepeatVector(output_sequence_length)
    logits = TimeDistributed(Dense(italian_vocab_size, activation='softmax'))
    
    model = Sequential()
    model.add(GRU(batch, input_shape = input_shape[1:]))
    
    model.add(repeatVector)
    model.add(rnn)
    model.add(logits)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
# OPTIONAL: Implement
tests.test_encdec_model(encdec_model)
print('tested')


# TODO: Reshape the input

tmp_x = pad(preproc_dutch_sentences, preproc_italian_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_italian_sentences.shape[-2], 1))


# TODO: Train the neural network
encdec_model= encdec_model(tmp_x.shape, preproc_italian_sentences.shape[1], dutch_vocab_size, italian_vocab_size)
encdec_model.fit(tmp_x, preproc_italian_sentences, batch_size=1024, epochs=10, validation_split=0.2)


C:\Users\France\Anaconda3\lib\site-packages\keras\engine\sequential.py:109: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


tested
Train on 116 samples, validate on 30 samples
Epoch 1/10
116/116 [==============================] - 12s 107ms/step - loss: 7.0022 - acc: 0.7814 - val_loss: 6.9422 - val_acc: 0.7962
Epoch 2/10
116/116 [==============================] - 3s 23ms/step - loss: 6.9433 - acc: 0.7814 - val_loss: 6.5273 - val_acc: 0.7962
Epoch 3/10
116/116 [==============================] - 3s 23ms/step - loss: 6.5350 - acc: 0.7814 - val_loss: 2.4388 - val_acc: 0.7962
Epoch 4/10
116/116 [==============================] - 3s 26ms/step - loss: 2.5024 - acc: 0.7814 - val_loss: 1.7901 - val_acc: 0.7962
Epoch 5/10
116/116 [==============================] - 3s 24ms/step - loss: 1.8650 - acc: 0.7814 - val_loss: 14.5133 - val_acc: 2.6882e-04
Epoch 6/10
116/116 [==============================] - 3s 23ms/step - loss: 14.4532 - acc: 0.0000e+00 - val_loss: 9.0307 - val_acc: 0.3398
Epoch 7/10
116/116 [==============================] - 3s 23ms/step - loss: 9.8526 - acc: 0.2841 - val_loss: 3.9395 - val_acc: 0.6605
Epoch

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [350]:
def model_final(input_shape, output_sequence_length, dutch_vocab_size, italian_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param dutch_vocab_size: Number of unique dutch words in the dataset
    :param italian_vocab_size: Number of unique italian words in the dataset
    :return: Keras model built, but not trained
    """
    units = 128
    learning_rate = 1e-3
    embedding=Embedding(dutch_vocab_size, units, input_length=input_shape[1])
    rnn = LSTM(units)
    rnn2 = LSTM(units, return_sequences=True)
    #print(input_shape[1])
    #print(italian_vocab_size)
    bidirectional=Bidirectional(rnn)
    bidirectional2=Bidirectional(rnn2)
    repeatVector=RepeatVector(output_sequence_length)
    logits = TimeDistributed(Dense(italian_vocab_size, activation='softmax'))
    
    model = Sequential()
    model.add(embedding)
    model.add(bidirectional)
    model.add(repeatVector)
    model.add(rnn2)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    # TODO: Implement
    print(model.summary())
    #plot_model(model, to_file='model.png', show_shapes=True)
    return model
tests.test_model_final(model_final)



print('Final Model Loaded')
# TODO: Train the final model


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_67 (Embedding)     (None, 15, 128)           25472     
_________________________________________________________________
bidirectional_65 (Bidirectio (None, 256)               263168    
_________________________________________________________________
repeat_vector_48 (RepeatVect (None, 21, 256)           0         
_________________________________________________________________
lstm_52 (LSTM)               (None, 21, 128)           197120    
_________________________________________________________________
time_distributed_216 (TimeDi (None, 21, 344)           44376     
Total params: 530,136
Trainable params: 530,136
Non-trainable params: 0
_________________________________________________________________
None
Final Model Loaded


C:\Users\France\Anaconda3\lib\site-packages\keras\engine\sequential.py:109: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


## Prediction (IMPLEMENTATION)

In [351]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed dutch data
    :param y: Preprocessed italian data
    :param x_tk: dutch tokenizer
    :param y_tk: italian tokenizer
    """
# TODO: Train neural network using model_final
    tmp_x = pad(preproc_dutch_sentences)
    print(tmp_x.shape)
    print(preproc_italian_sentences.shape[1])
    
    model= model_final(tmp_x.shape, preproc_italian_sentences.shape[1], dutch_vocab_size, italian_vocab_size)
    model.fit(tmp_x, preproc_italian_sentences, batch_size=2048, epochs=10, validation_split=0.2)

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = '3 tips om uw hoorapparaten te beschermen'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('3 tips om uw hoorapparaten te beschermen')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))




In [352]:
final_predictions(preproc_dutch_sentences, preproc_italian_sentences, dutch_tokenizer, italian_tokenizer)


100
(255, 100)
124
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_68 (Embedding)     (None, 100, 128)          148736    
_________________________________________________________________
bidirectional_67 (Bidirectio (None, 256)               263168    
_________________________________________________________________
repeat_vector_49 (RepeatVect (None, 124, 256)          0         
_________________________________________________________________
lstm_54 (LSTM)               (None, 124, 128)          197120    
_________________________________________________________________
time_distributed_217 (TimeDi (None, 124, 1550)         199950    
Total params: 808,974
Trainable params: 808,974
Non-trainable params: 0
_________________________________________________________________
None
Train on 204 samples, validate on 51 samples
Epoch 1/10
204/204 [==============================] - 35s 173ms/ste

In [320]:
final_predictions(preproc_dutch_sentences, preproc_italian_sentences, dutch_tokenizer, italian_tokenizer)

82
(146, 82)
124
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_60 (Embedding)     (None, 82, 256)           216832    
_________________________________________________________________
bidirectional_53 (Bidirectio (None, 512)               1050624   
_________________________________________________________________
repeat_vector_43 (RepeatVect (None, 124, 512)          0         
_________________________________________________________________
lstm_42 (LSTM)               (None, 124, 256)          787456    
_________________________________________________________________
time_distributed_205 (TimeDi (None, 124, 1099)         282443    
Total params: 2,337,355
Trainable params: 2,337,355
Non-trainable params: 0
_________________________________________________________________
None
Train on 116 samples, validate on 30 samples
Epoch 1/10
116/116 [==============================] - 32s 272ms/s

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

In [ ]:
!!python -m nbconvert *.ipynb

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?